Initial Attempt

In [7]:
import torch
import torch.nn as nn
import numpy as np

In [8]:

class AddNet(nn.Module):
    def __init__(self):
        super(AddNet, self).__init__()
        #TODO: explain simple linear equation facilitated
        self.output = nn.Linear(2, 1)
    
    def forward(self, x):
        return self.output(x)


In [9]:
def normalize(x):
    """Normalizes input values to range [0,1]."""
    return x / 100.0

def denormalize(x):
    """Denormalizes values back to original range."""
    return x * 100.0

In [10]:

def generate_training_data(num_samples):
    x1 = torch.randint(0, 101, (num_samples, 1), dtype=torch.float32)
    x2 = torch.randint(0, 101, (num_samples, 1), dtype=torch.float32)
    inputs = torch.cat((normalize(x1), normalize(x2)), dim=1)
    targets = normalize(x1 + x2)
    return inputs, targets


In [11]:
def train_add_net(model, num_epochs=500, batch_size=32):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    
    inputs, targets = generate_training_data(1000)
    
    for epoch in range(num_epochs):
        idx = torch.randperm(inputs.shape[0])[:batch_size]
        batch_inputs = inputs[idx]
        batch_targets = targets[idx]
        
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            print_weights_and_bias(model)


In [12]:
def inference(model, x1, x2):
    """Performs model inference with normalization handling."""
    model.eval()
    with torch.no_grad():
        x1_norm = normalize(torch.tensor([[float(x1)]], dtype=torch.float32))
        x2_norm = normalize(torch.tensor([[float(x2)]], dtype=torch.float32))
        inputs = torch.cat((x1_norm, x2_norm), dim=1)
        predicted_sum_norm = model(inputs)
        return denormalize(predicted_sum_norm).item()

In [13]:

def test_addition(model, x1, x2):
    """User-friendly testing function for a single pair of numbers."""
    predicted_sum = inference(model, x1, x2)
    actual_sum = x1 + x2
    
    print(f"\nTest Result:")
    print(f"Numbers: {x1} + {x2}")
    print(f"Predicted sum: {predicted_sum:.4f}")
    print(f"Actual sum: {actual_sum:.4f}")
    print(f"Error: {abs(predicted_sum - actual_sum):.4f}")
    
    return predicted_sum


In [14]:
def batch_test(model, num_samples):
    """Performs batch testing with random numbers."""
    print(f"\nBatch Testing ({num_samples} samples):")
    print("-" * 50)
    
    for _ in range(num_samples):
        x1 = np.random.uniform(0, 100)
        x2 = np.random.uniform(0, 100)
        test_addition(model, x1, x2)

In [15]:
def print_weights_and_bias(model):
    weights = model.output.weight.data.numpy()
    bias = model.output.bias.data.numpy()
    print(f"Weights: [{weights[0][0]:.4f}, {weights[0][1]:.4f}]")
    print(f"Bias: {bias[0]:.4f}\n")

In [16]:
model = AddNet()
train_add_net(model)
test_addition(model, 5.7, 3.2)
batch_test(model, 3)

Epoch [100/500], Loss: 0.0110
Weights: [0.7485, 0.7415]
Bias: 0.2872

Epoch [200/500], Loss: 0.0021
Weights: [0.9148, 0.9051]
Bias: 0.1008

Epoch [300/500], Loss: 0.0001
Weights: [0.9769, 0.9756]
Bias: 0.0262

Epoch [400/500], Loss: 0.0000
Weights: [0.9964, 0.9961]
Bias: 0.0045

Epoch [500/500], Loss: 0.0000
Weights: [0.9996, 0.9996]
Bias: 0.0004


Test Result:
Numbers: 5.7 + 3.2
Predicted sum: 8.9376
Actual sum: 8.9000
Error: 0.0376

Batch Testing (3 samples):
--------------------------------------------------

Test Result:
Numbers: 33.10817160653971 + 16.97264567487534
Predicted sum: 50.1029
Actual sum: 50.0808
Error: 0.0221

Test Result:
Numbers: 36.84003183450767 + 39.84632700999463
Predicted sum: 76.6983
Actual sum: 76.6864
Error: 0.0120

Test Result:
Numbers: 81.35259687272651 + 30.189587108521177
Predicted sum: 111.5412
Actual sum: 111.5422
Error: 0.0010
